# DengAi Disease Prediction
    Shoaib ul din

In [1]:
import pandas as pd
import numpy as np


from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn import linear_model

from sklearn.metrics import mean_absolute_error

from sklearn.preprocessing import StandardScaler

Importing the datasets

In [2]:
X_train = pd.read_csv(r"C:\Users\Shoaib ul din PC\Desktop\New folder\dengue_prediction\data\dengue_features_train.csv")
Y_train = pd.read_csv(r"C:\Users\Shoaib ul din PC\Desktop\New folder\dengue_prediction\data\dengue_labels_train.csv")
X_test = pd.read_csv(r"C:\Users\Shoaib ul din PC\Desktop\New folder\dengue_prediction\data\dengue_features_test.csv")
Y_test = pd.read_csv(r"C:\Users\Shoaib ul din PC\Desktop\New folder\dengue_prediction\data\submission_format.csv")

#### Data Pre Processing

In [3]:
def preProcessing(X, trainCheck):
    # We have only two cities SO converting those into booleans
    is_sj = X.city == 'sj'

    #Getting the month from date
    months = X.week_start_date.apply(getMonthFromDate)

    # Removing the coloumns
    X.drop(['city', 'year', 'weekofyear', 'week_start_date'], axis=1, inplace=True)
    
    if trainCheck:
        # Drop the rows with null values
        X.dropna(inplace=True)
    else:
        X.fillna(0,inplace=True)

    # Standardizing
    scaler = StandardScaler()
    X[X.columns] = scaler.fit_transform(X)

    # After Standarizing Adding the city
    X['is_sj'] = is_sj.loc[X.index]

    sliced_months = months.loc[X.index]
    # Get dummies data for month
    month_features = pd.get_dummies(sliced_months)
    X = X.join(month_features)
    
    return X


def getMonthFromDate(week_start_date):
    return int(week_start_date[5:7])

In [6]:
#Appying the preprocessing to the dataset
X_train = preProcessing(X_train, trainCheck = True)
X_test = preProcessing(X_test, trainCheck = False)

In [7]:
# Selecting y values
Y_train = Y_train.total_cases.loc[X_train.index]
Y_test = Y_test.total_cases.loc[X_test.index]

In [8]:
#Applying the linear model
clf = linear_model.Ridge(alpha = 44)
clf.fit(X_train, Y_train)
Y_pred = clf.predict(X_test)

In [9]:
#Getting the MAE
mean_absolute_error(Y_test, Y_pred)

21.65009884094446

In [10]:
#Converting the prediction into Int as per requirment of dataset
Y_pred = Y_pred.astype('int64')

In [11]:
#Importing the submission format
exe = pd.read_csv(r"C:\Users\Shoaib ul din PC\Desktop\New folder\dengue_prediction\data\submission_format.csv")

In [12]:
#Setting the predictions into total_cases
exe.total_cases = Y_pred

#Exporting the dataset into CSV for submission
export_excel = exe.to_csv(r'C:\Users\Shoaib ul din PC\Desktop\New folder\dengue_prediction\data\export_dataframe.csv', index = None, header=True)
